## Historical Option Pricing using Black-Scholes and IBKR API

In [1]:
# !pip install ibapi

DEPRECATION: Loading egg at c:\python311\lib\site-packages\ibapi-10.19.2-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [2]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import pandas as pd
import numpy as np
import datetime as dt
from scipy import stats
import threading
import time

Tickers define the list of stocks, whether they are Calls or Puts, their strikes, and a shared expiration date 

In [ ]:
tickers = ["FB","AMZN","CSCO"]
rights = ["C","C","P"]
strikes = [320,3380,60]
expiry = "20211001"

Because the API returns all historical data through the same callback function (historicalData), I use the reqId (Request ID) to keep the data organized:


Underlying Stock Data (reqId < 100): Requests with an ID lower than 100 are treated as standard price data for the stocks (FB, AMZN, CSCO). These are stored in the self.hist_data dictionary.


Implied Volatility Data (reqId >= 100): Requests with an ID of 100 or higher are treated as Implied Volatility data. These are stored in the self.impVol_data dictionary.

In [3]:
class TradeApp(EWrapper, EClient): 
    def __init__(self): 
        EClient.__init__(self, self) 
        self.hist_data = {}
        self.impVol_data = {}
        
    def historicalData(self, reqId, bar):
        if reqId < 100:
            if tickers[reqId] not in self.hist_data:
                self.hist_data[tickers[reqId]] = [{"Date":bar.date,"Open":bar.open,"High":bar.high,"Low":bar.low,"Close":bar.close}]
            else:
                self.hist_data[tickers[reqId]].append({"Date":bar.date,"Open":bar.open,"High":bar.high,"Low":bar.low,"Close":bar.close})
        else:
            if tickers[reqId-100] not in self.impVol_data:
                self.impVol_data[tickers[reqId-100]] = [{"Date":bar.date,"Open":bar.open,"High":bar.high,"Low":bar.low,"Close":bar.close}]
            else:
                self.impVol_data[tickers[reqId-100]].append({"Date":bar.date,"Open":bar.open,"High":bar.high,"Low":bar.low,"Close":bar.close})
            
        print("reqID:{}, date:{}, open:{}, high:{}, low:{}, close:{}".format(reqId,bar.date,bar.open,bar.high,bar.low,bar.close))
        
    def historicalDataEnd(self, reqId, start, end):
        super().historicalDataEnd(reqId, start, end)
        print("HistoricalDataEnd. ReqId:", reqId, "from", start, "to", end)
        if reqId < 100:
            self.hist_data[tickers[reqId]] = pd.DataFrame(self.hist_data[tickers[reqId]])
            self.hist_data[tickers[reqId]].set_index("Date",inplace=True)
        else:
            self.impVol_data[tickers[reqId-100]] = pd.DataFrame(self.impVol_data[tickers[reqId-100]])
            self.impVol_data[tickers[reqId-100]].set_index("Date",inplace=True)
        hist_data_event.set()

### Contract Helper:

Function that defines the stock contract

In [ ]:
def usTechStk(symbol,sec_type="STK",currency="USD",exchange="ISLAND"):
    contract = Contract()
    contract.symbol = symbol
    contract.secType = sec_type
    contract.currency = currency
    contract.exchange = exchange
    return contract 

### **Dynamic Inputs for Black-Scholes Pricing**

To calculate the theoretical "Fair Value" of an option, the script retrieves two critical dynamic inputs via the Interactive Brokers API:

#### **1. Underlying Stock Price (S) - `ADJUSTED_LAST`**

* **Role**: As a derivative, the option's value depends directly on the stock price.
* **Logic**: Call values rise as S increases, while Put values rise as S falls.
* **Adjusted Data**: Using "Adjusted" prices prevents errors in the model caused by artificial price gaps from stock splits or dividends.

#### **2. Implied Volatility (\sigma) - `OPTION_IMPLIED_VOLATILITY`**

* **Role**: Represents the market's expectation of future price fluctuations.
* **Impact**: Higher volatility increases the risk premium, raising the option's price (the "Vega" effect).

#### **Integration in Code**

The script merges these variables to reconstruct historical pricing:

* **S** provides the **intrinsic value**.
* **\sigma** determines the **time value** and risk premium.
* Both are passed into the `black_scholes` function  to generate the final theoretical price.

In [ ]:
def histData(req_num,contract,duration,candle_size):
    """extracts historical data"""
    app.reqHistoricalData(reqId=req_num, 
                          contract=contract,
                          endDateTime='',
                          durationStr=duration,
                          barSizeSetting=candle_size,
                          whatToShow='ADJUSTED_LAST',
                          useRTH=1,
                          formatDate=1,
                          keepUpToDate=0,
                          chartOptions=[])	


def impVolData(req_num,contract,duration,candle_size):
    """extracts historical data"""
    app.reqHistoricalData(reqId=req_num, 
                          contract=contract,
                          endDateTime='',
                          durationStr=duration,
                          barSizeSetting=candle_size,
                          whatToShow='OPTION_IMPLIED_VOLATILITY',
                          useRTH=1,
                          formatDate=1,
                          keepUpToDate=0,
                          chartOptions=[])

In [4]:
def websocket_con():
    app.run()
    
app = TradeApp()
app.connect(host='127.0.0.1', port=4002, clientId=23) #port 4002 for ib gateway paper trading/7497 for TWS paper trading
con_thread = threading.Thread(target=websocket_con, daemon=True)
con_thread.start()
time.sleep(1)



ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


In [ ]:
hist_data_event = threading.Event()

### **Sequential Data Acquisition Logic**

This loop serves as the execution engine, coordinating the retrieval of two distinct datasets for each ticker while managing the asynchronous nature of the IBKR API.

* **Iteration and Setup**: The script loops through the predefined `tickers` list (e.g., FB, AMZN, CSCO) and creates a formal `Contract` object for each symbol.
* **Synchronization**: Using `threading.Event()`, the script pauses execution with `.wait()` to ensure it doesn't request new data until the previous request is fully received and processed.
* **Prices (`histData`)**: Requests 200 days of daily **Adjusted Last** prices using a standard index (0, 1, 2) as the `reqId`.
* **Volatility (`impVolData`)**: Requests 200 days of **Implied Volatility** using a 100-offset `reqId` (100, 101, 102) to keep the data streams separate.


In [ ]:
for ticker in tickers:
    contract = usTechStk(ticker)
    hist_data_event.clear()
    histData(tickers.index(ticker),contract,'200 D', '1 day')
    hist_data_event.wait()
    hist_data_event.clear()
    impVolData(100+tickers.index(ticker),contract,'200 D', '1 day')
    hist_data_event.wait()

In [ ]:
#extract and store historical data in dataframe
historicalData = app.hist_data
impVolData = app.impVol_data

### **Black-Scholes Formula & Data Alignment**


The Black-Scholes formula is a mathematical model used to determine the fair price of European-style options. It relies on five key inputs:

* **Stock Price (S):** The current market value of the underlying asset (retrieved as `ADJUSTED_LAST`).
* **Strike Price (K):** The fixed price at which the option holder can buy or sell the stock.
* **Time to Maturity (T):** The time remaining until the option expires (calculated via the `dayCount` function).
* **Volatility (\sigma):** The expected price fluctuation (retrieved as `OPTION_IMPLIED_VOLATILITY`).
* **Risk-Free Rate (r):** The theoretical return on a risk-free investment (set to `0.03` or 3% in your code).


In [ ]:
#################Black Scholes Option pricing Model############################
def black_scholes(stock_price,strike_price,vol,time,rate,right="Call"):
    d1 = (np.log(stock_price/strike_price) + (rate + 0.5* vol**2)*time)/(vol*np.sqrt(time))
    d2 = (np.log(stock_price/strike_price) + (rate - 0.5* vol**2)*time)/(vol*np.sqrt(time))
    nd1 = stats.norm.cdf(d1)
    nd2 = stats.norm.cdf(d2)
    n_d1 = stats.norm.cdf(-1*d1)
    n_d2 = stats.norm.cdf(-1*d2)
    if right.capitalize()[0] == "C":
        return round((stock_price*nd1) - (strike_price*np.exp(-1*rate*time)*nd2),2)
    else:
        return round((strike_price*np.exp(-1*rate*time)*n_d2) - (stock_price*n_d1),2)

 
def dayCount(DF,expiry):
    """function to calculate days to maturity for each day in past"""
    DF["maturity"] = ((dt.datetime.strptime(expiry, "%Y%m%d") - pd.to_datetime(DF.index)).days)/365


The Black-Scholes formula implemented in the code calculates the theoretical "fair price" of an option by processing five primary financial inputs.

### **Python Implementation**


#### **1. Calculating d_1 and d_2**

These intermediate variables determine the probabilities used in the final price:

* **d_1 Calculation**:
`d1 = (np.log(stock_price/strike_price) + (rate + 0.5* vol**2)*time)/(vol*np.sqrt(time))`


$$d_1 = \frac{\ln(S/K) + (r + \frac{\sigma^2}{2})T}{\sigma\sqrt{T}}$$

* **d_2 Calculation**:
`d2 = (np.log(stock_price/strike_price) + (rate - 0.5* vol**2)*time)/(vol*np.sqrt(time))`


$$d_2 = \frac{\ln(S/K) + (r - \frac{\sigma^2}{2})T}{\sigma\sqrt{T}}$$


#### **2. Normal Cumulative Distribution (N)**

The code uses `stats.norm.cdf` to calculate the probability that the option will expire in-the-money:

* `nd1`, `nd2`: Probabilities for Call options.
* `n_d1`, `n_d2`: Probabilities for Put options (calculated as N(-d_1) and N(-d_2)).

#### **3. Final Pricing Logic**

The function uses a conditional `if` statement to return either a Call or Put price, rounded to two decimal places:

* **Call Option (`right="Call"`)**:
`return round((stock_price*nd1) - (strike_price*np.exp(-1*rate*time)*nd2),2)`

$$C = S \cdot N(d_1) - K e^{-rT} \cdot N(d_2)$$

* **Put Option (Else)**:
`return round((strike_price*np.exp(-1*rate*time)*n_d2) - (stock_price*n_d1),2)`

$$P = K e^{-rT} \cdot N(-d_2) - S \cdot N(-d_1)$$


### **Variable Mapping**

To connect the code to financial theory:

* **`stock_price` (S)**: The current price of the stock.
* **`strike_price` (K)**: The price at which the option can be exercised.
* **`vol` (\sigma)**: The Implied Volatility (extracted from IBKR).
* **`time` (T)**: The time remaining until expiration in years.
* **`rate` (r)**: The risk-free interest rate (hardcoded as `0.03` in the script).

In [ ]:
#calculate historical option price using Black Scholes Formula    
option_prices = {}
for ticker in tickers:
    local_symbol = ticker+" "+expiry[2:]+rights[tickers.index(ticker)]+str(strikes[tickers.index(ticker)])
    option_prices[local_symbol] = pd.DataFrame(columns=["Open","High","Low","Close"])
    dayCount(historicalData[ticker],expiry)
    dayCount(impVolData[ticker],expiry)
    historicalData[ticker] = historicalData[ticker][historicalData[ticker]["maturity"] > 0]
    impVolData[ticker] = impVolData[ticker][impVolData[ticker]["maturity"] > 0]
    for column in ["Open","High","Low","Close"]:
        option_prices[local_symbol][column] = black_scholes(historicalData[ticker][column],
                                                            strikes[tickers.index(ticker)],
                                                            impVolData[ticker][column],
                                                            historicalData[ticker]["maturity"],
                                                            0.03,
                                                            rights[tickers.index(ticker)])